Entradas: Month, GHI, Wind Speed, Zenith, Azimuth, Hour Sine, Hour Cosine 

Target: Angulo

Dados do ano de 2019 inteiro


In [6]:
import pandas as pd
import pvlib
import numpy as np 
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('data.csv') 
# otimo e theta calculados com pvlib
angulos = pd.read_csv('otimo_2.csv')
thetas = pd.read_csv('backtracking.csv')
# zenite e azimute do get solar position
solpos = pd.read_csv('solarposition.csv')

# Filtrar o df e pegar dados de 04/07 até 18/11 (5760 dados)
#df_mes = df[(df['datetime'] >= '2019-07-22 00:00:00') & (df['datetime'] <= '2019-11-18 23:30:00')].copy()
df_mes = df.copy()
display(df_mes)
angulos_mes = angulos.copy()
display(angulos_mes)
df_mes['Angulo'] = angulos_mes['angulo_otimo']
thetas_mes = thetas.copy()
display(thetas_mes)
solpos_mes = solpos.copy()
display(solpos_mes)

,Unnamed: 0,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Solar Zenith Angle,Wind Speed,Temperature,datetime,datetime_utc,datetime_local
0,0,2019,1,1,0,0,0,0,0,114.47,2.9,21.5,2019-01-01 00:00:00,2019-01-01 00:00:00+00:00,2018-12-31 22:00:00-02:00
1,1,2019,1,1,0,30,0,0,0,119.37,2.8,21.4,2019-01-01 00:30:00,2019-01-01 00:30:00+00:00,2018-12-31 22:30:00-02:00
2,2,2019,1,1,1,0,0,0,0,123.75,2.8,21.3,2019-01-01 01:00:00,2019-01-01 01:00:00+00:00,2018-12-31 23:00:00-02:00
3,3,2019,1,1,1,30,0,0,0,127.48,2.7,21.2,2019-01-01 01:30:00,2019-01-01 01:30:00+00:00,2018-12-31 23:30:00-02:00
4,4,2019,1,1,2,0,0,0,0,130.42,2.7,21.1,2019-01-01 02:00:00,2019-01-01 02:00:00+00:00,2019-01-01 00:00:00-02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,2019,12,31,21,30,32,151,45,85.18,2.8,24.4,2019-12-31 21:30:00,2019-12-31 21:30:00+00:00,2019-12-31 18:30:00-03:00
17516,17516,2019,12,31,22,0,0,0,0,91.58,3.0,23.3,2019-12-31 22:00:00,2019-12-31 22:00:00+00:00,2019-12-31 19:00:00-03:00
17517,17517,2019,12,31,22,30,0,0,0,97.66,3.0,22.6,2019-12-31 22:30:00,2019-12-31 22:30:00+00:00,2019-12-31 19:30:00-03:00
17518,17518,2019,12,31,23,0,0,0,0,103.55,3.0,22.0,2019-12-31 23:00:00,2019-12-31 23:00:00+00:00,2019-12-31 20:00:00-03:00


,Unnamed: 0,datetime,angulo_otimo,irradiancia
0,0,2019-01-01 00:00:00,15,0.000000
1,1,2019-01-01 00:30:00,15,0.000000
2,2,2019-01-01 01:00:00,15,0.000000
3,3,2019-01-01 01:30:00,15,0.000000
4,4,2019-01-01 02:00:00,15,0.000000
...,...,...,...,...
17515,17515,2019-12-31 21:30:00,55,146.347268
17516,17516,2019-12-31 22:00:00,15,0.000000
17517,17517,2019-12-31 22:30:00,15,0.000000
17518,17518,2019-12-31 23:00:00,15,0.000000


,Unnamed: 0,datetime_utc,tracker_theta
0,0,2019-01-01 00:00:00+00:00,0.000000
1,1,2019-01-01 00:30:00+00:00,0.000000
2,2,2019-01-01 01:00:00+00:00,0.000000
3,3,2019-01-01 01:30:00+00:00,0.000000
4,4,2019-01-01 02:00:00+00:00,0.000000
...,...,...,...
17515,17515,2019-12-31 21:30:00+00:00,9.925587
17516,17516,2019-12-31 22:00:00+00:00,0.000000
17517,17517,2019-12-31 22:30:00+00:00,0.000000
17518,17518,2019-12-31 23:00:00+00:00,0.000000


,datetime_utc,apparent_zenith,zenith,apparent_elevation,elevation,azimuth,equation_of_time
0,2019-01-01 00:00:00+00:00,114.476095,114.476095,-24.476095,-24.476095,228.129189,-3.195874
1,2019-01-01 00:30:00+00:00,119.374683,119.374683,-29.374683,-29.374683,222.677718,-3.205782
2,2019-01-01 01:00:00+00:00,123.756554,123.756554,-33.756554,-33.756554,216.419942,-3.215687
3,2019-01-01 01:30:00+00:00,127.491193,127.491193,-37.491193,-37.491193,209.279223,-3.225590
4,2019-01-01 02:00:00+00:00,130.431269,130.431269,-40.431269,-40.431269,201.247715,-3.235491
...,...,...,...,...,...,...,...
17515,2019-12-31 21:30:00+00:00,85.161122,85.330101,4.838878,4.669899,246.845935,-3.032026
17516,2019-12-31 22:00:00+00:00,91.579861,91.579861,-1.579861,-1.579861,243.925505,-3.041970
17517,2019-12-31 22:30:00+00:00,97.666672,97.666672,-7.666672,-7.666672,240.669099,-3.051912
17518,2019-12-31 23:00:00+00:00,103.550002,103.550002,-13.550002,-13.550002,237.004479,-3.061851


In [8]:
# Adicionando colunas de outros dfs ao df
colunas_desejadas = ['zenith', 'azimuth']
coluna = ['tracker_theta']
df_mes[colunas_desejadas] = solpos_mes[colunas_desejadas]
df_mes[coluna] = thetas_mes[coluna]
display(df_mes)


,Unnamed: 0,Year,Month,Day,Hour,Minute,DHI,DNI,GHI,Solar Zenith Angle,Wind Speed,Temperature,datetime,datetime_utc,datetime_local,Angulo,zenith,azimuth,tracker_theta
0,0,2019,1,1,0,0,0,0,0,114.47,2.9,21.5,2019-01-01 00:00:00,2019-01-01 00:00:00+00:00,2018-12-31 22:00:00-02:00,15,114.476095,228.129189,0.000000
1,1,2019,1,1,0,30,0,0,0,119.37,2.8,21.4,2019-01-01 00:30:00,2019-01-01 00:30:00+00:00,2018-12-31 22:30:00-02:00,15,119.374683,222.677718,0.000000
2,2,2019,1,1,1,0,0,0,0,123.75,2.8,21.3,2019-01-01 01:00:00,2019-01-01 01:00:00+00:00,2018-12-31 23:00:00-02:00,15,123.756554,216.419942,0.000000
3,3,2019,1,1,1,30,0,0,0,127.48,2.7,21.2,2019-01-01 01:30:00,2019-01-01 01:30:00+00:00,2018-12-31 23:30:00-02:00,15,127.491193,209.279223,0.000000
4,4,2019,1,1,2,0,0,0,0,130.42,2.7,21.1,2019-01-01 02:00:00,2019-01-01 02:00:00+00:00,2019-01-01 00:00:00-02:00,15,130.431269,201.247715,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,2019,12,31,21,30,32,151,45,85.18,2.8,24.4,2019-12-31 21:30:00,2019-12-31 21:30:00+00:00,2019-12-31 18:30:00-03:00,55,85.330101,246.845935,9.925587
17516,17516,2019,12,31,22,0,0,0,0,91.58,3.0,23.3,2019-12-31 22:00:00,2019-12-31 22:00:00+00:00,2019-12-31 19:00:00-03:00,15,91.579861,243.925505,0.000000
17517,17517,2019,12,31,22,30,0,0,0,97.66,3.0,22.6,2019-12-31 22:30:00,2019-12-31 22:30:00+00:00,2019-12-31 19:30:00-03:00,15,97.666672,240.669099,0.000000
17518,17518,2019,12,31,23,0,0,0,0,103.55,3.0,22.0,2019-12-31 23:00:00,2019-12-31 23:00:00+00:00,2019-12-31 20:00:00-03:00,15,103.550002,237.004479,0.000000


Tratamento dos dados: Criar Hour_sine e Hour_cosine 

In [10]:
from sklearn.preprocessing import MinMaxScaler

def calcula_seno(hora,minuto):
    hour_sine = np.sin((2*np.pi*(hora*60+minuto))/1440)
    return hour_sine

def calcula_cosseno(hora,minuto):
    hour_cosine = np.cos((2*np.pi*(hora*60+minuto))/1440)
    return hour_cosine

senos = []
cossenos = []

for i, row in df_mes.iterrows(): # row representa cada linha do df
    hour_sine = calcula_seno(row['Hour'],row['Minute'])
    senos.append(hour_sine)
    hour_cosine = calcula_cosseno(row['Hour'],row['Minute'])
    cossenos.append(hour_cosine)

df_mes['Hour Sine'] = senos
df_mes['Hour Cosine'] = cossenos


cols = ['Unnamed: 0','Year','Day','Hour','Minute','Temperature','datetime_utc','datetime_local','zenith'] #'Cloud Type','Fill Flag','Surface Albedo'
df_mes = df_mes.drop(cols,axis=1)
display(df_mes)

,Month,DHI,DNI,GHI,Solar Zenith Angle,Wind Speed,datetime,Angulo,azimuth,tracker_theta,Hour Sine,Hour Cosine
0,1,0,0,0,114.47,2.9,2019-01-01 00:00:00,15,228.129189,0.000000,0.000000,1.000000
1,1,0,0,0,119.37,2.8,2019-01-01 00:30:00,15,222.677718,0.000000,0.130526,0.991445
2,1,0,0,0,123.75,2.8,2019-01-01 01:00:00,15,216.419942,0.000000,0.258819,0.965926
3,1,0,0,0,127.48,2.7,2019-01-01 01:30:00,15,209.279223,0.000000,0.382683,0.923880
4,1,0,0,0,130.42,2.7,2019-01-01 02:00:00,15,201.247715,0.000000,0.500000,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,12,32,151,45,85.18,2.8,2019-12-31 21:30:00,55,246.845935,9.925587,-0.608761,0.793353
17516,12,0,0,0,91.58,3.0,2019-12-31 22:00:00,15,243.925505,0.000000,-0.500000,0.866025
17517,12,0,0,0,97.66,3.0,2019-12-31 22:30:00,15,240.669099,0.000000,-0.382683,0.923880
17518,12,0,0,0,103.55,3.0,2019-12-31 23:00:00,15,237.004479,0.000000,-0.258819,0.965926


In [11]:
df_mes.to_csv('df_mes.csv', index=False)